# Homework 6: Image Classification with Keras

The homework consists of two parts: theoretical part (7 pts) and coding part (16.5+10 pts).
 - All theoretical questions must be answered in your own words, do not copy-paste text from the internet. Points can be deducted for terrible formatting or incomprehensible English.
 - Code must be commented. If you use code you found online, you have to add the link to the source you used. There is no penalty for using outside sources as long as you convince us you understand the code.

**Note that coding part consists of two different notebooks.**

*Once completed zip the entire directory containing this exercise and upload it to https://courses.cs.ut.ee/2020/nn/spring/Main/Practices.*

**If you did this homework together with course mates, please write here their names (answers still have to be your own!).**

**Name(s):** fill this in if applicable

## Part 1: Theory Exercises

These theoretical questions are about the material covered in the lecture about "Recurrent Neural Networks".

**Task 1.1 (7pts):** Recurrent networks: basic truths and architectures

Which of the following is true about recurrent neural networks. In case of ”True” bring
an example (e.g. suffices to refer to a slide in lecture), in case of ”False”, explain.

1. A recurrent network always produces one output prediction per timestep.
2. It is possible to put a convolutional layer between the input and the recurrent layer.
3. The final layer (that makes the classification/regression) must be connected directly to the recurrent layer.
4. In all RNNs types seen in the lecture, output produced at timestep t can only depend on inputs received at timesteps 0 to t, and not on future timesteps.
5. All layers of a recurrent neural network are recurrent.
6. Only one layer of the recurrent network can be recurrent (two recurrent layers would make no sense).
7. The activations of the hidden nodes stay the same across all timesteps.
8. Different timesteps use the exact same weight matrices only in recurrent layers. This does not apply to other layers.
9. Different timesteps use the exact same weight matrices. This applies to all layers.
10. Gradient clipping is important to battle against gradients getting weaker and weaker.
11. LSTM networks deal with long term dependencies better than simple RNNs, because they have less parameters.
12. The gates in LSTM are opened or closed depending on the current input and current hidden state and it does not matter which timestep it currently is.
13. I have trained my network to predict the weather on the 8th day based on the weather of previous 7 days. I now have weather from only past 5 days available. I can still use the same network to predict weather tomorrow (accuracy might be bad, but we can use fewer inputs without changing the network).
14. I can train a recurrent network to receive input only at the first timestep and then produce arbitrarily long sequence of outputs.

## Practical Exercises

You've written a lot of code in this course to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of the course, though, we're going to leave behind your beautiful codebase and instead migrate to a popular deep learning framework: [Keras](https://keras.io/). Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. Keras has very well designed API, which allows for concise code and flexibility at the same time.

### Installation

#### Linux

To install Keras enter following on Conda command-line:
```
conda install keras
```

By default Keras uses Tensorflow backend. If you happen to have recent NVidia GPU on your machine, you might want to install GPU version of Tensorflow:
```
conda install tensorflow-gpu
```
When you import Keras in your code, you should see "Using TensorFlow backend." notice.

PS. If you have non-NVidia GPU, you might want to give a shot to [PlaidML](http://vertex.ai/blog/announcing-plaidml) Keras backend.

#### Windows

To install Keras enter following on Conda command-line:
```
pip install keras
```
By default Keras uses Tensorflow backend. If you happen to have recent NVidia GPU on your machine, you might want to install GPU version of Tensorflow:
```
pip install tensorflow-gpu
```
When you import Keras in your code, you should see "Using TensorFlow backend." notice.

#### In case you face some problems
If you see an error about tensorflow or that "Softmax does not have axis argument", run the following line:
```
pip install tensorflow==1.12.0
```

If you get an error with Numpy, make sure you have the last version by running:
```
pip intall -U numpy
```

When working with the notebook `Network.ipynb`, if you have a problem importing `from keras.applications.imagenet_utils import CLASS_INDEX`, then you can use `keras_applications`:
```
pip install keras_applications==1.0.7
```
and tehn replace the line
```
from keras.applications.imagenet_utils import CLASS_INDEX
```
with
```
from keras_applications.imagenet_utils import CLASS_INDEX
```

**Note:** if with all this you are still in trouble, you can create a new environment with the `requirements.txt` file. This file contains the required packages and has been tested with this notebook.
```
conda create  python=3.6 --name neural_networks_env
source activate neural_networks_env
conda install --file practice6/requirements.txt
```
then make sure that the kernel you use in jupyer is neural_networks_env

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Loading Data

Keras includes builtin support for datasets like MNIST, CIFAR, etc.
The first time you load a dataset it might take some time to download.

In [ ]:
from keras.datasets import cifar10

# Keras comes with built-in loaders for common datasets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# shorten dataset for quicker training
# feel free to comment those out if you have GPU
X_train = X_train[:25000]
y_train = y_train[:25000]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

### First model

Keras has two options for creating models: [sequential](https://keras.io/getting-started/sequential-model-guide/) and [functional API](https://keras.io/getting-started/functional-api-guide/). We are going to use functional API, because it matches well with computational graph model what neural networks really are. Also functional API allows branching, layer sharing and other advanced features we might need later.

Study the following code and see how we first create input tensor `x`, then apply layers to intermediate tensors until we produce tensor of probabilities `p`. Layers are implemented as [functors](https://www.daniweb.com/programming/software-development/threads/485098/functors-in-python) in Keras, meaning they are objects that can be called as functions. You can print out intermediate tensors `h` to see their shapes. Remember, here we just define the computational graph, no calculations are made yet. For more information see Keras documentation:
 - [Core layers](https://keras.io/layers/core/)
 - [Convolutional layers](https://keras.io/layers/convolutional/)
 - [Activations](https://keras.io/activations/)

Then the model is created with `x` as input and `p` as output. Cross-entropy loss is applied to the network with Adam optimizer and additional accuracy metric. `sparse_categorical_crossentropy` loss allows us to pass integer class values directly as targets and is potentially more efficient than `categorical_crossentropy`, which needs one-hot vectors as targets. For more information see Keras documentation:
 - [Losses](https://keras.io/losses/)
 - [Optimizers](https://keras.io/optimizers/)
 - [Metrics](https://keras.io/metrics/)

`model.summary()` prints out layer list with output shapes and is very useful for debugging the model. Also it shows number of trainable parameters, which is useful metric to estimate the capacity of your network.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, Flatten, Dense
from keras.optimizers import Adam

# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(32, 32, 3))
h = Conv2D(32, (7, 7), strides=(2, 2))(x)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(10)(h)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

**Task 2.1 (1pt):** Why accuracy cannot be used as a loss function?

**Your answer:** *fill this in*

### First training

Now we train our first model in Keras. Keras follows simple style similar to Scikit-learn, where a model has methods `fit()`, `predict()` and `evaluate()`. Notice how we use `validation_split` parameter to automatically produce validation set that is 4% of the whole dataset.

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_split=0.04)

`History` object returned by `model.fit()` can be used to plot learning curves.

In [ ]:
plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'])
plt.title('Accuracy')

**Task 2.2 (1pt):** Why training set accuracy is lower than validation set accuracy in the first epochs? Hint: training accuracy is an average over all training batches in epoch. When is validation accuracy calculated?

**Your answer:** *fill this in*

We can use model `predict()` method to produce predictions on given dataset.

In [ ]:
y_hat = model.predict(X_test)
y_pred = np.argmax(y_hat, axis=1)
print("Test set accuracy:", np.mean(y_pred == y_test[:, 0]))

But if we just want to evaluate the metrics and losses, it's easier to use `evaluate()` method.

In [ ]:
# IPython notebook just prints out the returned values - loss and accuracy
model.evaluate(X_test, y_test)

In [ ]:
# the same for training set
model.evaluate(X_train, y_train)

### Data Normalization

The above results were not very impressive. What are we missing? It turns out neural networks work much better with normalized inputs, which are around 0 (the most effective region of activation function).

We are going to try out three different normalization methods:
 - **centering** (subtracting the mean),
 - **standardization** (subtract mean and divide by standard deviation),
 - **minmax** (subtract min and divide by max-min).

For images it is sufficient to calculate statistcs for each color channel, e.g. you take mean over entire training set and also over all pixel positions. NB! The statistics (mean, standard deviation, min/max) must be always calculated on training set and the same values must be applied to validation and test set.

**Task 2.3 (3pts):** try different methods to normalize the images.

In [ ]:
##############################################################################
# TODO: Normalize pixel values by centering (subtract mean of each channel). #
##############################################################################
X_train_mean = None
X_test_mean = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

# redefine the layers to reinitialize weights
x = Input(shape=(32, 32, 3))
h = Conv2D(32, (7, 7), strides=(2, 2))(x)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(10)(h)
p = Activation('softmax')(h)

model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
history = model.fit(X_train_mean, y_train, batch_size=64, epochs=5, validation_split=0.04)

In [ ]:
##############################################################################
# TODO: Normalize pixel values by subtracting mean and dividing by standard  #
# deviation (for each channel).                                              #
##############################################################################
X_train_meanstd = None
X_test_meanstd = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

# redefine the layers to reinitialize weights
x = Input(shape=(32, 32, 3))
h = Conv2D(32, (7, 7), strides=(2, 2))(x)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(10)(h)
p = Activation('softmax')(h)

model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history = model.fit(X_train_meanstd, y_train, batch_size=64, epochs=5, validation_split=0.04)

In [ ]:
##############################################################################
# TODO: Normalize pixel values by scaling them between 0 and 1. (MinMax)     #
##############################################################################
X_train_minmax = None
X_test_minmax = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

# redefine the layers to reinitialize weights
x = Input(shape=(32, 32, 3))
h = Conv2D(32, (7, 7), strides=(2, 2))(x)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(10)(h)
p = Activation('softmax')(h)

model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])
history = model.fit(X_train_minmax, y_train, batch_size=64, epochs=5, validation_split=0.04)

**Task 2.4 (1pt):** which normalization worked the best? What do you think why?

**Your answer:** *fill this in*

In [ ]:
##############################################################################
# TODO: Assign the best data to X_train_norm and X_test_norm variables.      #
##############################################################################
X_train_norm = None
X_test_norm = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### More elaborate network

Create a network with following layers:
1. 3x3 convolution with 32 filters, stride 1, padding same
2. batch normalization
3. relu
4. 3x3 convolution with 32 filters, stride 1, padding valid
5. batch normalization
6. relu
7. max pooling 2x2
8. dropout 0.25
7. flatten
8. dense 100
5. batch normalization
6. relu
8. dropout 0.5
8. dense 10
9. softmax

You can consult Keras documentation for layer parameters:
 - [Convolutional layers](https://keras.io/layers/convolutional/)
 - [Pooling layers](https://keras.io/layers/pooling/)
 - [Normalization layers](https://keras.io/layers/normalization/)

Use the best data normalization from the previous step! You should get validation accuracy above 65%!

**Task 2.5 (1pt):** understanding padding

What is the difference between "padding same" and "padding valid"?

**Your answer:**

**Task 2.6 (0.5pt):** understanding dropout

In Keras, is the dropout probability interpreted as probability of keeping each node or probability of dropping?

**Your answer:**

**Task 2.7 (6pts):** create a network with given layers.

In [ ]:
from keras.layers import MaxPooling2D, BatchNormalization, Dropout

##############################################################################
# TODO: Create a network with given layers. Please keep the tensor naming,   #
#       we will need it later.                                               #
##############################################################################
x = None
c1 = None
b1 = None
a1 = None
c2 = None
b2 = None
a2 = None
p2 = None
d2 = None
f2 = None
h3 = None
b3 = None
a3 = None
d3 = None
z = None
p = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()

# feel free to use more epochs if you have GPU
history = model.fit(X_train_norm, y_train, batch_size=64, epochs=5, validation_split=0.04)

In [ ]:
plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'])
plt.title('Accuracy')

### Feature extraction

Model trained for classification can be used in other creative ways, for example finding similar images. The hidden layer activations can be thought of as features extracted from the images. As the final goal is to classifiy images into categories, we can assume that in the last layers the features of semantically similar images are similar. That means they are close to each other according to Euclidean distance.


When choosing which layer features to use `Dense(100)` layer seems like a good pick - it is positioned late in the network (close to output), meaning the features should reflect the semantic meaning relatively well. Also, it has sensible dimensions - 100. We create a new model that outputs not the classification, but the `Dense(100)` layer features. We can now use `predict()` to produce features of the entire test set.

Notice we are using the activation value before dropout and before relu activation function.

In [ ]:
# outputs are the features from Dense(100) layer of the previously trained model
# because we do not redefine the layers, weights remain the same
extract_model = Model(inputs=x, outputs=h3)
features = extract_model.predict(X_test_norm)
features.shape

**Task 2.8 (1pt):** calculate euclidean distance matrix between test set images

In [ ]:
##############################################################################
# TODO: Calculate euclidean distance matrix between test set images.         #
# You can either use code from the first homework or scipy.                  #
##############################################################################
dists = None
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Now we take first 10 images from the test set and plot their 9 most similar images from the same set, along with distance.

In [ ]:
plt.figure(figsize=(16,16))
for i in range(10):
    closest = np.argsort(dists[i])[:10]
    for k, j in enumerate(closest):
        plt.subplot(10, 10, i*10 + k + 1)
        plt.imshow(X_test[j])
        plt.axis('off')
        plt.title("d=%0.2f" % dists[i, j])
plt.tight_layout()

**Task 2.9 (2pts):** Describe the results. Describe the mistakes it makes. A nearest-neighbour search on the input images would simply return images with most similar pixel values, how are these results different?  

**Your answer:** *fill this in*